In [1]:
import pandas as pd
from tqdm import tqdm

df = pd.read_excel('../predict_result/0927到1105測試結果.xlsx')
print(df.shape)
df.head()

(1040, 36)


,45A,項次,電文代號,銀行,20,回傳代碼,回傳訊息,建檔日期,受益人,金額,...,預測開狀人,利用公司名稱預測公司代號,DIVSION,DIVSION預測代號,集成預測代號,銀行輸入,開狀銀行,正確與否,錯誤原因,信用狀代碼(LCNO)
0,48 MT PVC SUSPENSION RESIN GRADE S65AT THE RA...,8,mt700,16,0006MLC00034022,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:31:17,FORMOSA PLASTICS CORPORATION,"USD136080,00",...,DEEP JYOTI WAX TRADERS,1,not find,not find,11,1. SIGNED COMMERCIAL INVOICE IN TWO ORIGINAL.\...,ICICI BANK LTD,yes,無錯誤,0006MLC00047821
1,TETRAHYDROFURAN AT USD 5550 TOTAL AMOUNTUSD 1...,10,mt700,16,001LM01212660007,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:32:17,NAN YA PLASTICS CORPORATION,"USD111000,",...,TETRAHYDROFURAN AT USD 5550 TOTAL AMOUNTUSD 1...,2,not find,not find,24,+1.SIGNED COMMERCIAL INVOICES IN 6 COPIES FOR ...,YES BANK LIMITED,yes,無錯誤,594LM01210050004
2,A 19800 MT OF TAIRILAC AG15A1HABS ABS RESINAT...,11,mt700,16,0393MLC00068522,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:33:05,FORMOSA CHEMICALS AND FIBRECORPORATION,"USD736110,00",...,SUPREME PETROCHEM LIMITED,4,CHEMICALS,2P,4A,"1.SIGNED COMMERCIAL INVOICE IN 3 COPIES, CER...",ICICI BANK LTD,yes,無錯誤,001LM01211750013
3,80 MT PVC SUSPENSION RESIN GRADE S65S AT THE ...,12,mt700,16,0505121IM0000553,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:01:28,FORMOSA PLASTICS CORPORATION,"USD154710,00",...,RMG POLYVINYL INDIA LIMITED,1,not find,not find,11,1. DRAFT FOR 100 PERCENT OF THE INVOICE VALUE....,MEGA INTERNATIONAL COMMERCIAL,yes,無錯誤,0172FOSU2115003
4,PVC SUSPENSION RESIN S70RQUANTITY 72 MT UNIT ...,13,mt700,16,0799M21SA0000380,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:01:04,FORMOSA PLASTICS CORPORATION,"USD152640,",...,KTM KIMYEVI MADDELER ITHALAT,1,not find,not find,11,1. COMMERCIAL INVOICE IN 2 ORIGINALS AND 2 COP...,CITITWTX,yes,無錯誤,0799M20SA0000268


In [2]:
df.columns

Index(['45A', '項次', '電文代號', '銀行', '20', '回傳代碼', '回傳訊息', '建檔日期', '受益人', '金額',
       '推薦公司事業部', '是否已轉MIS', 'MEMO', 'from', '50', '59', '46A', '47A', '78',
       'memo', 'Unnamed: 5', '預測產品', '預測產品(取長度最長)', '預測產品使用方式', '根據產品預測部門',
       '根據產品預測代號', '預測開狀人', '利用公司名稱預測公司代號', 'DIVSION', 'DIVSION預測代號', '集成預測代號',
       '銀行輸入', '開狀銀行', '正確與否', '錯誤原因', '信用狀代碼(LCNO)'],
      dtype='object')

In [3]:
def save_color_df(df,save_path,x_cols=['45A','50','59','銀行輸入'],y_cols=['預測產品(取長度最長)','預測開狀人','受益人','開狀銀行']):
    # 建立writer
    writer = pd.ExcelWriter(save_path, engine='xlsxwriter')
    # 將 df 第一個 row 變成欄位名稱
    new_df = pd.DataFrame()
    for i in df.columns:
        new_df[i] = [i] + df[i].values.tolist() 
    df = new_df
    # 存檔
    df.to_excel(writer, sheet_name='Sheet1', header=False, index=False)
    # 參數設定
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    cell_format_red = workbook.add_format({'font_color': 'red'})
    cell_format_default = workbook.add_format({'bold': False})
    worksheet.write_row('A1',df.columns.tolist())
    # 定義反紅功能函數
    def add_word_color(df,x_col,y_col):
        for row in range(0,len(df)):
            word = df.iloc[row,:][y_col]
            detect_col_idx = df.columns.tolist().index(x_col)
            try:
                # 1st case, wrong word is at the start and there is additional text
                if (df.iloc[row,detect_col_idx].index(word) == 0) \
                and (len(df.iloc[row,detect_col_idx]) != len(word)):
                    worksheet.write_rich_string(row,detect_col_idx,cell_format_red,
                        word,cell_format_default,df.iloc[row,detect_col_idx][len(word):])

                # 2nd case, wrong word is at the middle of the string
                elif (df.iloc[row,detect_col_idx].index(word) > 0) \
                and (df.iloc[row,detect_col_idx].index(word) != len(df.iloc[row,detect_col_idx])-len(word)) \
                and ('Typo:' not in df.iloc[row,detect_col_idx]):
                    starting_point = df.iloc[row,detect_col_idx].index(word)
                    worksheet.write_rich_string(row, detect_col_idx, cell_format_default,
                                        df.iloc[row,detect_col_idx][0:starting_point],
                                        cell_format_red, word, cell_format_default,
                                        df.iloc[row,detect_col_idx][starting_point+len(word):])

                # 3rd case, wrong word is at the end of the string
                elif (df.iloc[row,detect_col_idx].index(word) > 0) \
                and (df.iloc[row,detect_col_idx].index(word) == len(df.iloc[row,detect_col_idx])-len(word)):
                    starting_point = df.iloc[row,detect_col_idx].index(word)
                    worksheet.write_rich_string(row, detect_col_idx, cell_format_default,
                                                df.iloc[row,detect_col_idx][0:starting_point],
                                                cell_format_red, word)

                # 4th case, wrong word is the only one in the string
                elif (df.iloc[row,detect_col_idx].index(word) == 0) \
                and (len(df.iloc[row,detect_col_idx]) == len(word)):
                    worksheet.write(row, detect_col_idx, word, cell_format_red)

            except ValueError:
                continue
    
    # 執行多次反紅功能函數
    for x,y in zip(x_cols,y_cols):
        add_word_color(df,x,y)
    
    # 存檔
    writer.save()

In [4]:
for fname in tqdm(sorted([ i for i in df['from'].unique()])):
    d = df.loc[df['from']==fname]
    save_name = '../data/測試結果/'+fname[:4]+'測試結果.xlsx'
    save_color_df(d,save_name)
    print(save_name,d.shape)
    

  4%|▍         | 1/25 [00:00<00:06,  3.50it/s]

../data/測試結果/0927測試結果.xlsx (34, 36)


  8%|▊         | 2/25 [00:00<00:06,  3.55it/s]

../data/測試結果/0928測試結果.xlsx (56, 36)


 12%|█▏        | 3/25 [00:00<00:05,  3.74it/s]

...

../data/測試結果/1029測試結果.xlsx (54, 36)
../data/測試結果/1101測試結果.xlsx (44, 36)


 92%|█████████▏| 23/25 [00:05<00:00,  4.10it/s]

../data/測試結果/1102測試結果.xlsx (65, 36)
../data/測試結果/1103測試結果.xlsx (40, 36)


100%|██████████| 25/25 [00:05<00:00,  4.46it/s]

../data/測試結果/1104測試結果.xlsx (50, 36)
../data/測試結果/1105測試結果.xlsx (35, 36)
